# CONSTRUCCIÓN DE RECURSOS LÉXICOS PARA QUECHUA

## Objetivo
Extraer, estructurar y enriquecer el diccionario bilingüe Quechua-Español para crear un corpus léxico machine-readable.

## Estructura del Proyecto
- **Entrada**: Diccionario PDF bilingüe (Quechua-Español / Español-Quechua)
- **Salida**: Archivos JSON estructurados + librería Python para consultas

## Tareas
1. Extracción de texto crudo del PDF
2. Identificación y separación de secciones
3. Diseño e implementación de parsers
4. Generación de archivos JSON estructurados
5. Desarrollo de librería de utilidades
6. Validación y testing

In [1]:
# Instalación de librerías necesarias
# !pip install PyMuPDF pdfplumber regex

# Importaciones
import fitz  # PyMuPDF
import pdfplumber
import re
import json
import os
from typing import List, Dict, Optional, Tuple
from collections import defaultdict
import pandas as pd

print("Librerías instaladas correctamente")

Librerías instaladas correctamente


In [2]:
# TAREA 1 y 2: Extracción de texto del PDF y separación de secciones

def extraer_texto_pdf(ruta_pdf: str) -> str:
    """
    Extrae texto crudo del PDF preservando estructura básica sin alteraciones
    """
    texto_completo = ""
    
    try:
        # Usar PyMuPDF para extracción
        doc = fitz.open(ruta_pdf)
        
        for pagina_num in range(doc.page_count):
            pagina = doc[pagina_num]
            texto = pagina.get_text()
            texto_completo += texto
        
        doc.close()
        
    except Exception as e:
        print(f"Error con PyMuPDF: {e}")
        
        # Alternativa con pdfplumber
        try:
            with pdfplumber.open(ruta_pdf) as pdf:
                for pagina in pdf.pages:
                    texto = pagina.extract_text()
                    if texto:
                        texto_completo += texto
        except Exception as e2:
            print(f"Error con pdfplumber: {e2}")
            return ""
    
    return texto_completo

def guardar_texto_crudo(texto: str, ruta_salida: str = "diccionario_raw.txt"):
    """
    Guarda el texto crudo extraído del PDF
    """
    with open(ruta_salida, 'w', encoding='utf-8') as f:
        f.write(texto)
    print(f"Texto crudo guardado en: {ruta_salida}")

# Ejecutar extracción
ruta_pdf = "diccionario-qeswa-academia-mayor.pdf"
if os.path.exists(ruta_pdf):
    print("Extrayendo texto del PDF...")
    texto_crudo = extraer_texto_pdf(ruta_pdf)
    guardar_texto_crudo(texto_crudo)
    print(f"Texto extraído: {len(texto_crudo)} caracteres")
else:
    print(f"Archivo PDF no encontrado: {ruta_pdf}")
    print("Por favor, asegúrese de que el archivo esté en el directorio correcto")

Extrayendo texto del PDF...
Texto crudo guardado en: diccionario_raw.txt
Texto extraído: 1930001 caracteres
Texto crudo guardado en: diccionario_raw.txt
Texto extraído: 1930001 caracteres


In [8]:
# TAREA 3: Extracción y limpieza de secciones del diccionario

def limpiar_encabezados_y_patrones(texto: str) -> str:
    """
    Limpia el texto eliminando encabezados de página, títulos, letras de sección y otros patrones no deseados.
    """
    lineas = texto.split('\n')
    lineas_limpias = []
    
    for linea in lineas:
        linea_original = linea
        linea = linea.strip()
        
        # Ignorar líneas vacías
        if not linea:
            continue
        
        # PATRONES A ELIMINAR:
        
        # 1. Números de página junto con DICCIONARIO
        if re.match(r'^\d+$', linea) or 'DICCIONARIO' in linea.upper():
            continue
        
        # 2. Símbolos decorativos como ◄●►
        if re.match(r'^[◄●►\-=\*\+\s]+$', linea):
            continue
        
        # 3. Título SIMI TAQE
        if 'SIMI TAQE' in linea.upper():
            continue
        
        # 4. Letras sueltas de sección (una sola letra mayúscula en su propia línea)
        if re.match(r'^[A-Z]$', linea):
            continue
        
        # 5. Encabezados típicos de PDF
        if any(patron in linea.upper() for patron in [
            'PÁGINA', 'CAPÍTULO', 'SECCIÓN', 'ÍNDICE', 'CONTENIDO',
            'DICCIONARIO QUECHUA', 'ESPAÑOL - QUECHUA', 'QUECHUA - ESPAÑOL'
        ]):
            continue
        
        # 6. Líneas que son solo números o símbolos
        if re.match(r'^[\d\s\-\.\*\+◄●►]+$', linea):
            continue
        
        # 7. Líneas muy cortas que no parecen ser contenido útil (menos de 3 caracteres)
        if len(linea) < 3:
            continue
        
        # 8. Patrones específicos de encabezados de página
        if re.match(r'^\d+\s*◄●►\s*$', linea) or re.match(r'^◄●►\s*\d+\s*$', linea):
            continue
        
        # Si la línea pasó todos los filtros, mantenerla
        lineas_limpias.append(linea_original)
    
    return '\n'.join(lineas_limpias)

def separar_secciones_por_lineas(texto: str) -> Tuple[str, str]:
    """
    Separa las secciones usando los números de línea conocidos y aplica limpieza de encabezados
    """
    lineas = texto.split('\n')
    
    # Sección Quechua-Español: línea 1325 a 50998
    seccion_qe_lineas = lineas[1325:50998]
    
    # Limpiar texto extra al final de la sección Quechua-Español
    seccion_qe_contenido = []
    for linea in seccion_qe_lineas:
        if "ESPAÑOL - QUECHUA" in linea:
            break
        seccion_qe_contenido.append(linea)
    
    # Sección Español-Quechua: línea 50998 hasta el final
    seccion_eq_contenido = []
    for i in range(50998, len(lineas)):
        linea = lineas[i].strip()
        if ("DICCIONARIO QUECHUA – ESPAÑOL – QUECHUA" in linea or 
            "se terminó de imprimir" in linea or 
            "talleres gráficos" in linea):
            break
        seccion_eq_contenido.append(lineas[i])
    
    # Aplicar limpieza de encabezados y patrones no deseados
    seccion_qe_texto = '\n'.join(seccion_qe_contenido)
    seccion_eq_texto = '\n'.join(seccion_eq_contenido)
    
    seccion_qe_limpia = limpiar_encabezados_y_patrones(seccion_qe_texto)
    seccion_eq_limpia = limpiar_encabezados_y_patrones(seccion_eq_texto)
    
    return seccion_qe_limpia, seccion_eq_limpia

# Ejecutar extracción y limpieza de secciones
if os.path.exists("diccionario_raw.txt"):
    with open("diccionario_raw.txt", 'r', encoding='utf-8') as f:
        texto_completo = f.read()
    
    print("Procesando secciones del diccionario...")
    
    seccion_qe, seccion_eq = separar_secciones_por_lineas(texto_completo)
    
    # Contar líneas aproximadas para verificar el contenido
    lineas_qe = [l for l in seccion_qe.split('\n') if l.strip()]
    lineas_eq = [l for l in seccion_eq.split('\n') if l.strip()]
    
    print(f"Sección Quechua-Español: {len(lineas_qe)} líneas")
    print(f"Sección Español-Quechua: {len(lineas_eq)} líneas")
    
    # Guardar archivos limpios
    with open("seccion_quechua_espanol.txt", 'w', encoding='utf-8') as f:
        f.write(seccion_qe)
    
    with open("seccion_espanol_quechua.txt", 'w', encoding='utf-8') as f:
        f.write(seccion_eq)
    
    print("Archivos guardados correctamente")
    
else:
    print("Error: archivo diccionario_raw.txt no encontrado")

Procesando secciones del diccionario...
Sección Quechua-Español: 48883 líneas
Sección Español-Quechua: 9264 líneas
Archivos guardados correctamente
Sección Quechua-Español: 48883 líneas
Sección Español-Quechua: 9264 líneas
Archivos guardados correctamente


In [11]:
# TAREA 4: Diseño e implementación de parsers

def extraer_entradas_completas(texto: str) -> List[str]:
    """
    Extrae entradas completas del texto, incluyendo las que abarcan múltiples líneas.
    Mejorado para manejar lemas compuestos (ej: "Amaru Inka Yupanki") y apostrofes (ej: "map'a")
    CORRIGE: Problema de absorción de lemas siguientes con remisiones V.
    """
    lineas = texto.split('\n')
    entradas = []
    entrada_actual = ""
    
    # Patrón mejorado que incluye apostrofes y detecta inicios de entrada
    patron_inicio = r'^[a-zA-ZÀ-ÿñÑ!¡¿?,\s\']+[\.!]\s+(s\.|adj\.|v\.|adv\.|interj\.|prep\.|conj\.|pron\.|alfab\.|loc\.)'
    
    for linea in lineas:
        linea = linea.strip()
        
        # Verificar si es el inicio de una nueva entrada
        if re.match(patron_inicio, linea):
            # Guardar entrada anterior si existe
            if entrada_actual.strip():
                entradas.append(entrada_actual.strip())
            # Iniciar nueva entrada
            entrada_actual = linea
        else:
            # Continuar entrada actual
            if entrada_actual:
                entrada_actual += " " + linea
    
    # Agregar la última entrada
    if entrada_actual.strip():
        entradas.append(entrada_actual.strip())
    
    # POST-PROCESAMIENTO: Separar entradas que fueron incorrectamente unidas
    entradas_corregidas = []
    
    for entrada in entradas:
        # Buscar patrones de lemas que fueron absorbidos incorrectamente
        # Patrón: texto completo + lema. categoria V. REFERENCIA
        patron_absorcion = r'^(.+?)\s+([a-zA-ZÀ-ÿñÑ!¡¿?,\s\']+[\.!])\s+(V\.\s+[A-ZÁÉÍÓÚÑ\s]+\.?)\s*$'
        match = re.match(patron_absorcion, entrada)
        
        if match:
            # Separar las entradas
            entrada_principal = match.group(1).strip()
            lema_absorbido = match.group(2).strip()
            remision = match.group(3).strip()
            
            # Verificar que el lema absorbido es realmente un lema válido
            if re.match(r'^[a-zA-ZÀ-ÿñÑ!¡¿?,\s\']+[\.!]$', lema_absorbido):
                entradas_corregidas.append(entrada_principal)
                entradas_corregidas.append(f"{lema_absorbido} {remision}")
            else:
                entradas_corregidas.append(entrada)
        else:
            entradas_corregidas.append(entrada)
    
    return entradas_corregidas

def parsear_entrada_ultra_estricta(entrada_texto: str, abreviaturas: Dict, es_espanol: bool = False) -> List[Dict]:
    """
    Parser ULTRA-ESTRICTO mejorado según todas las especificaciones del usuario.
    Maneja: lemas compuestos, remisiones (V.), definiciones múltiples, || como separador de acepciones/dialectales.
    """
    if len(entrada_texto) < 10:
        return []
    
    # 1. EXTRAER LEMA (puede ser compuesto: "Amaru Inka Yupanki")
    # Patrón mejorado para lemas compuestos
    match_lema = re.match(r'^([a-zA-ZÀ-ÿñÑ!¡¿?,\s]+?)[\.!]\s', entrada_texto)
    if not match_lema:
        return []
    
    lema = match_lema.group(1).strip()
    # Mantener mayúsculas para lemas compuestos (ej: "Amaru Inka Yupanki")
    # Solo convertir a minúscula si es una sola palabra que no empieza con mayúscula
    if ' ' not in lema and not lema[0].isupper():
        lema = lema.lower()
    
    # Extraer resto del texto después del lema
    resto = entrada_texto[len(match_lema.group(0)):].strip()
    
    # Filtro para español: evitar palabras quechuas
    if es_espanol:
        if any(c in lema.lower() for c in ['k', 'w', 'q']) and 'qu' not in lema.lower():
            return []
    
    # Obtener listas de abreviaturas
    categorias_gramaticales = abreviaturas.get('categorias_gramaticales', [])
    campos_semanticos = abreviaturas.get('campos_semanticos', [])
    dialectales = abreviaturas.get('dialectales', [])
    
    # 2. EXTRAER CATEGORÍA GRAMATICAL (inmediatamente después del lema)
    categoria = ''
    for cat in sorted(categorias_gramaticales, key=len, reverse=True):
        if resto.startswith(cat):
            categoria = cat
            resto = resto[len(cat):].strip()
            break
    
    if not categoria:
        return []
    
    # 3. DETECTAR TODOS LOS CAMPOS SEMÁNTICOS EN TODA LA DEFINICIÓN (REGLA 1)
    campos_encontrados = []
    texto_busqueda = resto
    
    # Buscar TODOS los campos semánticos en todo el texto, no solo al inicio
    for campo in sorted(campos_semanticos, key=len, reverse=True):
        # Buscar en diferentes posiciones: inicio, después de punto, en medio
        patrones_busqueda = [
            f'^{re.escape(campo)}\\s',          # Al inicio
            f'\\s{re.escape(campo)}\\s',        # En medio del texto
            f'\\.\\s*{re.escape(campo)}\\s',    # Después de punto
            f'^{re.escape(campo)}\\.',          # Al inicio con punto
            f'\\s{re.escape(campo)}\\.'         # En medio con punto
        ]
        
        for patron in patrones_busqueda:
            if re.search(patron, texto_busqueda):
                if campo not in campos_encontrados:
                    campos_encontrados.append(campo)
                break
    
    # 4. PROCESAMIENTO MEJORADO DE CONTENIDO DESPUÉS DE ||
    if '||' in resto:
        partes_separadas = [a.strip() for a in resto.split('||') if a.strip()]
    else:
        partes_separadas = [resto]
    
    # Estructura única de entrada que iremos completando
    entrada = {
        'lema': lema,
        'categoria_gramatical': [categoria],  # Lista para múltiples categorías
        'campo_semantico': campos_encontrados.copy(),
        'definicion': '',
        'variantes_dialectales': {},
        'sinonimos': [],
        'ejemplos': []
    }
    
    # Lista para acumular todas las definiciones
    todas_definiciones = []
    
    for i, parte in enumerate(partes_separadas):
        texto = parte.strip()
        
        # 5. DETECTAR REMISIONES "V. LEMA" (véase)
        match_remision = re.search(r'\bV\.\s+([A-ZÁÉÍÓÚÑ][A-ZÁÉÍÓÚÑ\s]*)', texto)
        if match_remision:
            remite_a = match_remision.group(1).strip().lower()
            entrada['remite_a'] = remite_a
            texto = re.sub(r'\bV\.\s+[A-ZÁÉÍÓÚÑ][A-ZÁÉÍÓÚÑ\s]*', '', texto).strip()
        
        # 6. DETECTAR CATEGORÍA GRAMATICAL ADICIONAL (para casos como "|| adj. insult.")
        categoria_adicional = ''
        for cat in sorted(categorias_gramaticales, key=len, reverse=True):
            if texto.startswith(cat):
                categoria_adicional = cat
                if cat not in entrada['categoria_gramatical']:
                    entrada['categoria_gramatical'].append(cat)
                texto = texto[len(cat):].strip()
                break
        
        # 7. DETECTAR CAMPOS SEMÁNTICOS ADICIONALES (para casos como "|| Agri.")
        campos_adicionales = []
        for campo in sorted(campos_semanticos, key=len, reverse=True):
            if texto.startswith(campo):
                campos_adicionales.append(campo)
                if campo not in entrada['campo_semantico']:
                    entrada['campo_semantico'].append(campo)
                texto = texto[len(campo):].strip()
                break
        
        # 8. DETERMINAR TIPO DE CONTENIDO DESPUÉS DE ||
        es_dialectal = False
        for dialecto in dialectales:
            dialecto_sin_punto = dialecto.rstrip('.')
            if (f'{dialecto_sin_punto}:' in texto or 
                texto.startswith(dialecto_sin_punto + ':') or
                f' {dialecto_sin_punto}:' in texto):
                es_dialectal = True
                break
        
        # 9. EXTRAER EJEMPLOS (EJEM:)
        if 'EJEM:' in texto:
            pos_ejem = texto.find('EJEM:')
            texto_antes = texto[:pos_ejem].strip()
            texto_ejem = texto[pos_ejem + 5:].strip()
            
            # Encontrar final del ejemplo (hasta dialecto o final)
            fin_ejemplo = len(texto_ejem)
            for dialecto in dialectales:
                dialecto_sin_punto = dialecto.rstrip('.')
                pos_dialecto = texto_ejem.find(f'{dialecto_sin_punto}:')
                if pos_dialecto != -1 and pos_dialecto < fin_ejemplo:
                    fin_ejemplo = pos_dialecto
            
            ejemplo_texto = texto_ejem[:fin_ejemplo].strip(' .,!')
            if ejemplo_texto and ejemplo_texto not in entrada['ejemplos']:
                entrada['ejemplos'].append(ejemplo_texto)
            
            # Continuar con el resto después del ejemplo
            resto_ejemplo = texto_ejem[fin_ejemplo:].strip()
            texto = texto_antes + ' ' + resto_ejemplo
            texto = texto.strip()
        
        # 8. EXTRAER SINÓNIMOS (SINÓN:) 
        if 'SINÓN:' in texto:
            pos_sinon = texto.find('SINÓN:')
            texto_antes = texto[:pos_sinon].strip()
            texto_sinon = texto[pos_sinon + 6:].strip()
            
            # Los sinónimos terminan en el PRIMER punto seguido de mayúscula O dialecto
            fin_sinon = len(texto_sinon)
            
            # Buscar primer punto seguido de mayúscula
            for j, char in enumerate(texto_sinon):
                if char == '.' and j + 1 < len(texto_sinon):
                    siguiente = texto_sinon[j + 1:j + 10].strip()
                    if siguiente and siguiente[0].isupper():
                        fin_sinon = j + 1
                        break
            
            # Buscar abreviaturas dialectales que interrumpen sinónimos
            for dialecto in dialectales:
                dialecto_sin_punto = dialecto.rstrip('.')
                for patron in [f' {dialecto_sin_punto}:', f' {dialecto_sin_punto} ']:
                    pos = texto_sinon.find(patron)
                    if pos != -1 and pos < fin_sinon:
                        fin_sinon = pos
            
            sinon_texto = texto_sinon[:fin_sinon].strip(' .,!')
            if sinon_texto:
                sinonimos = [s.strip() for s in sinon_texto.split(',') if s.strip()]
                entrada['sinonimos'] = [re.sub(r'[^\w\sñÑáéíóúÁÉÍÓÚ]', '', s).strip() 
                                      for s in sinonimos if len(s.strip()) > 1]
            
            # Continuar con el resto después de los sinónimos
            resto_despues_sinon = texto_sinon[fin_sinon:].strip()
            texto = texto_antes + ' ' + resto_despues_sinon
            texto = texto.strip()
        
        # 11. EXTRAER VARIANTES DIALECTALES (MEJORADO PARA ||)
        texto_limpio = texto
        dialectales_ordenados = sorted(dialectales, key=len, reverse=True)
        
        for dialecto in dialectales_ordenados:
            dialecto_sin_punto = dialecto.rstrip('.')
            
            # Buscar patrón más flexible: "Arg:", " Arg:", al inicio o después de espacios
            patrones_dialectal = [
                f'{dialecto_sin_punto}:',      # Al inicio: "Arg:"
                f' {dialecto_sin_punto}:',     # En medio: " Arg:"
                f'^{dialecto_sin_punto}:'      # Exactamente al inicio
            ]
            
            pos_encontrada = -1
            patron_usado = ''
            
            for patron in patrones_dialectal:
                if patron.startswith('^'):
                    # Patrón de inicio exacto
                    if texto_limpio.startswith(patron[1:]):
                        pos_encontrada = 0
                        patron_usado = patron[1:]
                        break
                else:
                    pos = texto_limpio.find(patron)
                    if pos != -1:
                        pos_encontrada = pos
                        patron_usado = patron
                        break
            
            if pos_encontrada != -1:
                texto_antes = texto_limpio[:pos_encontrada].strip()
                texto_despues = texto_limpio[pos_encontrada + len(patron_usado):].strip()
                
                # Encontrar final de variantes dialectales
                fin_dialectal = len(texto_despues)
                
                # Buscar próximo dialecto
                for otro_dialecto in dialectales_ordenados:
                    if otro_dialecto != dialecto:
                        otro_sin_punto = otro_dialecto.rstrip('.')
                        for patron_otro in [f'{otro_sin_punto}:', f' {otro_sin_punto}:']:
                            pos_otro = texto_despues.find(patron_otro)
                            if pos_otro != -1 and pos_otro < fin_dialectal:
                                fin_dialectal = pos_otro
                
                # Buscar marcadores que interrumpen
                for marcador in [' SINÓN:', ' EJEM:', '. SINÓN:', '. EJEM:']:
                    pos_marcador = texto_despues.find(marcador)
                    if pos_marcador != -1 and pos_marcador < fin_dialectal:
                        fin_dialectal = pos_marcador
                
                contenido_dialectal = texto_despues[:fin_dialectal].strip(' .,!')
                
                if contenido_dialectal:
                    # ANALIZAR CONTENIDO: formato "Anc: Caj: pujllana" o "achala, achocha"
                    regiones_adicionales = []
                    variantes_finales = []
                    
                    # Separar por dos puntos para detectar regiones comprimidas
                    partes_dos_puntos = contenido_dialectal.split(':')
                    
                    if len(partes_dos_puntos) > 1:
                        # Hay formato comprimido: "Anc: Caj: pujllana"
                        for parte in partes_dos_puntos[:-1]:  # Todas excepto la última
                            parte = parte.strip()
                            # Verificar si es una región válida (3-4 letras, empieza con mayúscula)
                            if re.match(r'^[A-Z][a-z]{2,3}$', parte):
                                regiones_adicionales.append(parte)
                        
                        # La última parte son las variantes
                        variantes_texto = partes_dos_puntos[-1].strip()
                        variantes_finales = [v.strip() for v in variantes_texto.split(',') if v.strip()]
                    else:
                        # Formato simple: "achala, achocha"
                        variantes_finales = [v.strip() for v in contenido_dialectal.split(',') if v.strip()]
                    
                    # CREAR ESTRUCTURA DE VARIANTES DIALECTALES (SIN "Gen")
                    if '.' in dialecto and len(dialecto.split('.', 1)[1].rstrip('.')) > 0:
                        # Caso: Pe.Aya. → pais='Pe', region='Aya'
                        pais, region_principal = dialecto.split('.', 1)
                        region_principal = region_principal.rstrip('.')
                        
                        if pais not in entrada['variantes_dialectales']:
                            entrada['variantes_dialectales'][pais] = {}
                        
                        # Agregar región principal
                        entrada['variantes_dialectales'][pais][region_principal] = variantes_finales
                        
                        # Agregar regiones adicionales con las mismas variantes
                        for region_adicional in regiones_adicionales:
                            entrada['variantes_dialectales'][pais][region_adicional] = variantes_finales
                            
                    else:
                        # Caso: Arg. → pais='Arg', sin estructura anidada
                        pais = dialecto.rstrip('.')
                        entrada['variantes_dialectales'][pais] = variantes_finales
                
                # Limpiar del texto principal
                texto_limpio = texto_antes + ' ' + texto_despues[fin_dialectal:]
                texto_limpio = texto_limpio.strip()
        
        # 10. EXTRAER DEFINICIÓN (lo que queda después de limpiar todo)
        definicion = texto_limpio
        
        # REGLA 1: Limpiar TODOS los campos semánticos detectados de la definición
        for campo in campos_encontrados:
            # Limpiar campo al inicio de la definición
            definicion = re.sub(f'^{re.escape(campo)}\\s*', '', definicion)
            # Limpiar campo en medio de la definición
            definicion = re.sub(f'\\s*{re.escape(campo)}\\s*', ' ', definicion)
            # Limpiar campo después de punto
            definicion = re.sub(f'\\.\\s*{re.escape(campo)}\\s*', '. ', definicion)
        
        # Limpiar campos adicionales encontrados después de ||
        for campo in campos_adicionales:
            definicion = re.sub(f'^{re.escape(campo)}\\s*', '', definicion)
            definicion = re.sub(f'\\s*{re.escape(campo)}\\s*', ' ', definicion)
        
        # Limpiar marcadores residuales
        definicion = re.sub(r'SINÓN:.*$', '', definicion)
        definicion = re.sub(r'EJEM:.*$', '', definicion)
        definicion = re.sub(r'VARIEDADES:.*$', '', definicion, flags=re.IGNORECASE)
        definicion = re.sub(r'\bV\.\s+[A-ZÁÉÍÓÚÑ][A-ZÁÉÍÓÚÑ\s]*', '', definicion)
        
        # Limpiar dialectales residuales
        for dialecto in dialectales:
            dialecto_sin_punto = dialecto.rstrip('.')
            definicion = re.sub(f'{re.escape(dialecto_sin_punto)}:.*$', '', definicion)
        
        # 12. EXTRAER DEFINICIÓN (lo que queda después de limpiar todo)
        definicion_parte = texto_limpio
        
        # Limpiar campos semánticos del texto
        for campo in entrada['campo_semantico']:
            definicion_parte = re.sub(f'^{re.escape(campo)}\\s*', '', definicion_parte)
            definicion_parte = re.sub(f'\\s*{re.escape(campo)}\\s*', ' ', definicion_parte)
            definicion_parte = re.sub(f'\\.\\s*{re.escape(campo)}\\s*', '. ', definicion_parte)
        
        # Limpiar marcadores residuales
        definicion_parte = re.sub(r'SINÓN:.*$', '', definicion_parte)
        definicion_parte = re.sub(r'EJEM:.*$', '', definicion_parte)
        definicion_parte = re.sub(r'VARIEDADES:.*$', '', definicion_parte, flags=re.IGNORECASE)
        definicion_parte = re.sub(r'^\([^)]*\)\.\s*', '', definicion_parte)  # Nombres científicos solos
        
        # REGLA 3: Evitar contaminación - detectar nueva entrada
        patron_nueva_entrada = r'\b[a-zA-Z]+\.\s+(s\.|adj\.|v\.|adv\.|interj\.|prep\.|conj\.|pron\.|alfab\.|loc\.|\bV\.)'
        match_nueva = re.search(patron_nueva_entrada, definicion_parte)
        if match_nueva:
            definicion_parte = definicion_parte[:match_nueva.start()].strip()
        
        definicion_parte = re.sub(r'\s+', ' ', definicion_parte).strip(' .,!')
        
        # AGREGAR A DEFINICIONES TOTALES
        # Primera parte siempre se incluye, partes después de || se incluyen si no son puramente dialectales
        if i == 0:
            # Primera parte - siempre incluir
            if definicion_parte:
                todas_definiciones.append(definicion_parte)
        else:
            # Partes después de || - incluir si hay contenido y no es solo dialectal
            if definicion_parte and not es_dialectal:
                todas_definiciones.append(definicion_parte)
            elif definicion_parte and es_dialectal:
                # Si es dialectal pero hay contenido antes del dialecto, incluirlo
                texto_antes_dialectos = definicion_parte
                for dialecto in dialectales:
                    dialecto_sin_punto = dialecto.rstrip('.')
                    patron_corte = f'{dialecto_sin_punto}:'
                    if patron_corte in texto_antes_dialectos:
                        pos_corte = texto_antes_dialectos.find(patron_corte)
                        texto_antes_dialectos = texto_antes_dialectos[:pos_corte].strip()
                        break
                if texto_antes_dialectos:
                    todas_definiciones.append(texto_antes_dialectos)
        
    # COMBINAR TODAS LAS DEFINICIONES
    if todas_definiciones:
        entrada['definicion'] = '. '.join(todas_definiciones)
    
    # FORMATEAR CATEGORÍA GRAMATICAL
    if len(entrada['categoria_gramatical']) == 1:
        entrada['categoria_gramatical'] = entrada['categoria_gramatical'][0]
    
    # FORMATEAR CAMPO SEMÁNTICO - mantener como lista vacía si no hay campos
    if not entrada['campo_semantico']:
        entrada['campo_semantico'] = []
    
    # RETORNAR ENTRADA ÚNICA (ya no necesitamos múltiples entradas por lema)
    if (entrada['definicion'] or entrada['sinonimos'] or 
        entrada['variantes_dialectales'] or entrada.get('remite_a')):
        return [entrada]
    else:
        return []

class QuechuaEspanolParser:
    """
    Parser específico para la sección Quechua-Español
    Utiliza el parser ultra-estricto que separa correctamente los campos
    """
    
    def __init__(self, abreviaturas: Dict[str, List[str]]):
        self.abreviaturas = abreviaturas
    
    def parsear_seccion(self, texto: str, max_entradas: int = 10000) -> List[Dict]:
        """
        Parsea la sección Quechua-Español con el parser ultra-estricto
        """
        entradas = []
        
        # Extraer entradas completas (multilinea)
        entradas_texto = extraer_entradas_completas(texto)
        print(f"Encontradas {len(entradas_texto)} entradas potenciales")
        
        contador = 0
        for entrada_texto in entradas_texto:
            if contador >= max_entradas:
                break
                
            entradas_procesadas = parsear_entrada_ultra_estricta(entrada_texto, self.abreviaturas, es_espanol=False)
            if entradas_procesadas:
                entradas.extend(entradas_procesadas)
                contador += len(entradas_procesadas)
        
        return entradas

class EspanolQuechuaParser:
    """
    Parser específico para la sección Español-Quechua
    Utiliza el parser ultra-estricto que separa correctamente los campos
    """
    
    def __init__(self, abreviaturas: Dict[str, List[str]]):
        self.abreviaturas = abreviaturas
    
    def parsear_seccion(self, texto: str, max_entradas: int = 10000) -> List[Dict]:
        """
        Parsea la sección Español-Quechua con el parser ultra-estricto
        """
        entradas = []
        
        # Extraer entradas completas (multilinea)
        entradas_texto = extraer_entradas_completas(texto)
        print(f"Encontradas {len(entradas_texto)} entradas potenciales")
        
        contador = 0
        for entrada_texto in entradas_texto:
            if contador >= max_entradas:
                break
                
            entradas_procesadas = parsear_entrada_ultra_estricta(entrada_texto, self.abreviaturas, es_espanol=True)
            if entradas_procesadas:
                entradas.extend(entradas_procesadas)
                contador += len(entradas_procesadas)
        
        return entradas


In [14]:
# TAREA 5: Generación de archivos JSON estructurados

import time

def cargar_abreviaturas():
    """
    Carga las abreviaturas desde el archivo JSON generado anteriormente
    """
    try:
        with open("abreviaturas.json", 'r', encoding='utf-8') as f:
            abreviaturas = json.load(f)
        return abreviaturas
    except FileNotFoundError:
        print("Error: archivo abreviaturas.json no encontrado")
        return {
            'categorias_gramaticales': ['s.', 'adj.', 'v.', 'adv.', 'interj.', 'prep.', 'conj.', 'pron.'],
            'campos_semanticos': ['Bot.', 'Zool.', 'Med.', 'Hist.', 'Geog.'],
            'dialectales': ['Arg.', 'Bol.', 'Pe.Anc.', 'Pe.Aya.', 'Pe.Qos.']
        }

def generar_archivos_json():
    """
    Ejecuta los parsers y genera los archivos JSON según las especificaciones
    """
    # Cargar abreviaturas
    abreviaturas = cargar_abreviaturas()

    # Crear parsers usando las clases de la Tarea 4
    parser_qe = QuechuaEspanolParser(abreviaturas)
    parser_eq = EspanolQuechuaParser(abreviaturas)
    
    # Procesar sección Quechua-Español
    entradas_qe = []
    if os.path.exists("seccion_quechua_espanol.txt"):
        print("\nProcesando sección Quechua-Español...")
        with open("seccion_quechua_espanol.txt", 'r', encoding='utf-8') as f:
            texto_qe = f.read()
        
        entradas_qe = parser_qe.parsear_seccion(texto_qe, max_entradas=15000)
        print(f"{len(entradas_qe)} entradas procesadas")
    else:
        print("Archivo seccion_quechua_espanol.txt no encontrado")
    
    # Procesar sección Español-Quechua
    entradas_eq = []
    if os.path.exists("seccion_espanol_quechua.txt"):
        print("\nProcesando sección Español-Quechua...")
        with open("seccion_espanol_quechua.txt", 'r', encoding='utf-8') as f:
            texto_eq = f.read()
        
        entradas_eq = parser_eq.parsear_seccion(texto_eq, max_entradas=15000)
        print(f"{len(entradas_eq)} entradas procesadas")
    else:
        print("Archivo seccion_espanol_quechua.txt no encontrado")
    
    # Guardar archivos JSON
    if entradas_qe:
        with open("quechua_espanol.json", 'w', encoding='utf-8') as f:
            json.dump(entradas_qe, f, indent=2, ensure_ascii=False)
        print(f"\nGuardado: quechua_espanol.json ({len(entradas_qe)} entradas)")
    
    if entradas_eq:
        with open("espanol_quechua.json", 'w', encoding='utf-8') as f:
            json.dump(entradas_eq, f, indent=2, ensure_ascii=False)
        print(f"Guardado: espanol_quechua.json ({len(entradas_eq)} entradas)")
    



generar_archivos_json()



Procesando sección Quechua-Español...
Encontradas 16277 entradas potenciales
Encontradas 16277 entradas potenciales
9978 entradas procesadas

Procesando sección Español-Quechua...
Encontradas 4577 entradas potenciales
9978 entradas procesadas

Procesando sección Español-Quechua...
Encontradas 4577 entradas potenciales
4571 entradas procesadas
4571 entradas procesadas

Guardado: quechua_espanol.json (9978 entradas)

Guardado: quechua_espanol.json (9978 entradas)
Guardado: espanol_quechua.json (4571 entradas)
Guardado: espanol_quechua.json (4571 entradas)
